In [33]:
import os
import tensorflow as tf
import numpy as np
import shutil
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [34]:
# Diretórios
classificacao_dir = 'image/classificacao'
treinamento_dir = 'image/treinamento'
classes = ["boa", "ruim"]

In [35]:
# Criar diretórios de treinamento se não existirem
for classe in classes:
    os.makedirs(os.path.join(treinamento_dir, classe), exist_ok=True)

In [36]:
# Função para emparelhar imagens (assume nomes como "solda1_frente.jpg" e "solda1_tras.jpg")
def emparelhar_imagens(origem_dir):
    imagens = sorted([f for f in os.listdir(origem_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])
    pares = {}

    for img in imagens:
        nome_base = img.rsplit('_', 1)[0]  # Remove "_frente" ou "_tras"
        if nome_base not in pares:
            pares[nome_base] = []
        pares[nome_base].append(os.path.join(origem_dir, img))

    return [p for p in pares.values() if len(p) == 2]  # Apenas pares completos

In [37]:
# Copiar imagens para diretórios de treinamento
for classe in classes:
    origem = os.path.join(classificacao_dir, classe)
    destino = os.path.join(treinamento_dir, classe)
    pares = emparelhar_imagens(origem)

    for par in pares:
        for img in par:
            shutil.copy(img, destino)

print("Imagens copiadas para treinamento!")

Imagens copiadas para treinamento!


In [38]:
# Função para processar e concatenar pares de imagens
def processar_par_de_imagens(par):
    img1 = cv2.imread(par[0])
    img2 = cv2.imread(par[1])

    img1 = cv2.resize(img1, (128, 128))
    img2 = cv2.resize(img2, (128, 128))

    img_concatenada = np.concatenate((img1, img2), axis=1)  # Junta lado a lado
    return img_concatenada

In [39]:
# Criar gerador customizado para carregar pares de imagens
class ParImageDataGenerator:
    def __init__(self, directory, batch_size=32, target_size=(128, 256), class_mode='sparse'):
        self.directory = directory
        self.batch_size = batch_size
        self.target_size = target_size
        self.class_mode = class_mode
        self.classes = classes
        self.data = self._load_data()

    def _load_data(self):
        data = []
        for classe in self.classes:
            classe_dir = os.path.join(self.directory, classe)
            pares = emparelhar_imagens(classe_dir)
            for par in pares:
                data.append((par, self.classes.index(classe)))
        return data

    def __iter__(self):
        return self

    def __next__(self):
        if not self.data:
            raise StopIteration
        batch = self.data[:self.batch_size]
        self.data = self.data[self.batch_size:]

        inputs = [processar_par_de_imagens(par) for par, _ in batch]
        labels = [label for _, label in batch]

        return np.array(inputs) / 255.0, np.array(labels)

In [40]:
# Criar geradores de treinamento e validação
train_generator = ParImageDataGenerator(treinamento_dir)
validation_generator = ParImageDataGenerator(treinamento_dir)

In [41]:
# Caminho do modelo
modelo_path = 'model/modelo_solda.h5'

In [42]:
# Verifica se o modelo já existe
if os.path.exists(modelo_path):
    model = tf.keras.models.load_model(modelo_path)
    print("Modelo carregado, continuando o treinamento...")
else:
    print("Treinando um novo modelo...")

    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 256, 3)),  # Largura dobrada
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Treinando um novo modelo...


In [43]:
# **Verificar se há dados antes de treinar**
try:
    x_train, y_train = next(iter(train_generator))
    x_val, y_val = next(iter(validation_generator))

    model.fit(
        x=x_train, y=y_train,
        epochs=5,
        validation_data=(x_val, y_val)
    )

    # Salvar o modelo treinado
    model.save(modelo_path)
    print(f"Modelo salvo em: {modelo_path}")

except StopIteration:
    print("Erro: Nenhum dado disponível para treino. Verifique as imagens no diretório.")

Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 0.7006 - val_accuracy: 0.9375 - val_loss: 0.2455
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step - accuracy: 0.9375 - loss: 0.2455 - val_accuracy: 0.4375 - val_loss: 2.9138
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 511ms/step - accuracy: 0.4375 - loss: 2.9138 - val_accuracy: 0.9375 - val_loss: 0.0734
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.9375 - loss: 0.0734 - val_accuracy: 0.5625 - val_loss: 1.3314
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step - accuracy: 0.5625 - loss: 1.3314 - val_accuracy: 0.9375 - val_loss: 0.2414


Modelo salvo em: model/modelo_solda.h5
